In [1]:
from numpy import *
from exact_r import *

### An augmented solver for the homogenous shallow water equations

\begin{equation}
 h_{t} + (hu)_x = 0 \\
 (hu)_t + \left(hu^{2} + \frac{1}{2}gh^{2} \right)_x = 0
\end{equation}

LeVeque and Pelanti decomposition is given by:
\begin{equation}
 \begin{bmatrix} H_{i} - H_{i-1} \\ HU_{i} - HU_{i-1} \\ \phi(Q_{i}) -  \phi(Q_{i-1})\end{bmatrix} = \sum_{p=1}^{3} \alpha_{i-\frac{1}{2}}^{p} \mathcal{w}_{i-\frac{1}{2}}^{p}
\end{equation}


## Initial conditions

The solver should supply an initialization routine to initialize  q(x,t)  at time  t=0 .


In [6]:
def h_init(x,hl,hr):    
    q0 = where(x < 0,hl,hr)
    return q0

def hu_init(x,hl,ul,hr,ur):    
    #q0 = zeros(x.shape)  
    q0 = where(x<0,hl*ul,hr*ur)
    return q0

def qinit(x,meqn):
    
    q = zeros((x.shape[0],meqn))
    q[:,0] = h_init(x,hl,hr)
    q[:,1] = hu_init(x,hl,ul,hr,ur)
    
    return q

In [7]:
#flux
def flux(q):
    '''
    input:
    -----
    q - state at the interface
    return:
    -------
    f - flux at the interface
    '''
    q1 = q[0]
    q2 = q[1]
    f = zeros(2)
    f[0] = q2
    f[1] = (((q2)**2)/q1) + (0.5*g*(q1)**2)
    return f

In [8]:
#momentum flux
def phi(q,g):
    h = q[0]
    u = q[1]/q[0]
    return (h*u**2 + (1/2)*g*(h**2))

In [13]:
#Determine the wave type in each family (Riemann sturcture)
def riemanntype(hl,hr,ul,ur,drytol,newton,maxiter,g):
    
    #test for the wave type
    hmin = min(hr,hl)
    hmax = max(hr,hl)
    delu = ur - ul
    
    if hmin <= drytol:
        hm = 0
        um = 0
        #eigen values of the Roe averaged Jacobian
        s1m = ur + ul - 2*sqrt(g*hr) + 2*sqrt(g*hl)
        s2m = ur + ul - 2*sqrt(g*hr) + 2*sqrt(g*hl)
        
        if hl <= 0:
            rare2 = True
            rare1 = False
        else:
            rare1 = True
            rare2 = False
        
    else:
        fmin = delu + 2*sqrt(g*hmin) - sqrt(g*hmax)
        fmax = delu + (hmax - hmin)*sqrt(0.5*g*(hmax + hmin)/(hmax*hmin))
        
        #2-rarefactions
        if fmin >= 0:
            hm = (1/(16*g))*max(0,-delu + 2*(sqrt(g*hl) + sqrt(g*hr)))**2
            um = copysign(1,hm)*(ul + 2*(sqrt(g*hl) - sqrt(g*hm)))
            
            s1m = ul + 2*sqrt(g*hl) - 3*sqrt(g*hm)
            s2m = ur - 2*sqrt(g*hr) + 3*sqrt(g*hm)
            
            rare1 = True
            rare2 = True
        
        #2-shocks
        elif fmax <= 0:
            hm,um = newton(ql,qr,g)
            u1m = ul - (hm-hl)*sqrt((0.5*g)*(1/hm + 1/hl))
            u2m = ur + (hm - hr)*sqrt((0.5*g)*(1/hm + 1/hr))
            
            s1m = u1m - sqrt(g*hm)
            s2m = u2m + sqrt(g*hm)
            
            rare1 = False
            rare2 = False
        
        # 1-shock 1-rarefaction
        else:
            ho = hmin
            
            for i in range(maxiter):
                fo = delu + 2*(sqrt(g*ho) - sqrt(g*hmax)) \
                + (ho - hmin)*sqrt(0.5*g*(1/ho + 1/hmin))
                
                slope = (fmax - fo)/(hmax-hmin)
                ho -= fo/slope
            
            hm = ho
            if hl <= hr:
                um = ul + 2*sqrt(g*hl) - 2*sqrt(g*hm)
                s1m = ul + 2*sqrt(g*hl) - 3*sqrt(g*hm)
                s2m = ul + 2*sqrt(g*hl) - sqrt(g*hm)
                rare1 = True
                rare2 False
                
            else:
                s1m = ur - 2*sqrt(g*hr) + sqrt(g*hm)
                s2m = ur - 2*sqrt(g*hr) + 3*sqrt(g*hm)
                um = ur - 2*sqrt(g*hr) + 2*sqrt(g*hm)
                rare2 = True
                rare1 = False
    
    qm = array([hm,hm*um])
    return qm,s1m,s2m,rare1,rare2
    

In [53]:
def riemann_JCP(ql,qr,drytol,newton,maxiter,g,sE1,sE2,phiR,phiL,\
                mwaves,meqn,bL,bR,vL,vR):
    
    #left height and velocity field
    hl = ql[0]
    hul = ql[1]
    ul = ql[1]/ql[0]
    
    #right height and velocity field
    hr = qr[0]
    hur = qr[1]
    ur = qr[1]/qr[0]
    
    #determine del vectors
    delh = hr - hl
    delhu = hur - hul
    delphi = phiR - phiL
    delb = bR - bl
    delnorm = delh**2 + delphi**2
    
    #call function riemanntype
    qm,s1m,s2m,rare1,rare2 = riemanntype(hl,hr,ul,ur,drytol,newton,maxiter,g)
    hm = qm[0]
    um = qm[1]
    
    #modified Einfeldt speed
    lam = zeros(3)
    lam[0] = min(sE1,s2m)
    lam[1] = 0 
    lam[2] = max(sE2,s1m)
    sE1 = lam[0]
    sE2 = lam[2]
    
    #middle state in an HLL solver
    hstar = max((hul - hur + sE2*hr - sE1*hl)/(sE2-sE1),0)
    
    #determine the middle entropy corrector wave
    rarecorrectortest = False
    rarecorrector = False
    if rarecorrectortest:
        sdelta = lam[2] - lam[0]
        raremin = 0.5
        raremax = 0.9
        if rare1 and sE1*s1m < 0:
            raremin = 0.2
        if rare2 and sE2*s2m < 0:
            raremin = 0.2
        if rare1 or rare2:
            #check w/c rarefaction is larger
            rare1st = 3*(sqrt(g*hl) - sqrt(g*hm))
            rare2st = 3*(sqrt(g*hr) - sqrt(g*hm))
            if max(rare1st,rare2st) > raremin*sdelta and \
                max(rare1st,rare2st) < raremax*sdelta:
                rarecorrector = True
                if rare1st > rare2st:
                    lam[1] = s1m
                elif rare2st > rare1st:
                    lam[1] = s2m
                else:
                    lam[1] = (0.5)*(s1m + s2m)
        
    if hstar < min(hl,hr)/5:
        rarecorrector = False
    
    r = zeros((3,3))
    for mw in range(mwaves):
        r[0,mw] = 1
        r[1,mw] = lam[mw]
        r[2,mw] = (lam[mw])**2
    
    if not rarecorrector:
        lam[1] = (0.5)*(lam[0] + lam[2])
        r[0,1] = 0
        r[1,1] = 0
        r[2,1] = 1
        
    #steady state wave
    criticaltol = 1e-6
    deldelh = -delb
    deldelphi = -g*0.5*(hr + hl)*delb
    
    #Determine a few quantities needed for steady state wave if iterated
    hlstar = hl
    hrstar = hr
    ulstar = ul
    urstar = ur
    hulstar = ulstar*hlstar
    hurstar = urstar*hrstar
    
    #Iterate to better determine the steady state wave
    convergencetol = 1e-6
    for i in range(maxiter):
        #determine steady state wave
        if min(hlstar,hrstar) < drytol and rarecorrector:
            rarecorrector = False
            hlstar = hl
            hrstar = hr
            ulstar = ul
            urstar = ur
            hulstar = ulstar*hlstar
            hurstar = urstar*hrstar
            lam[1] = 0.5*(lam[0] + lam[2])
            lam[1] = max(min(0.5*(s1m + s2m),sE2),sE1)
            r[0,1] = 0
            r[1,1] = 0 
            r[2,1] = 1
        
        #average depth
        hbar = max(0.5*(hlstar + hrstar),0)
        #product of shallow water eigen values
        s1s2bar = 0.25*(ulstar + urstar)**2 - g*hbar
        s1s2tilde = max(0,ulstar*urstar) - g*hbar
        
        #find if sonic problem
        sonic = False
        if abs(s1s2bar <= criticaltol):
            sonic = True
        if s1s2bar*s1s2tilde <= criticaltol:
            sonic = True
        if s1s2bar*sE1*sE2 <= criticaltol:
            sonic = True
        if min(abs(sE1),abs(sE2)) < criticaltol:
            sonic = True
        if sE1 < 0 and s1m > 0:
            sonic = True
        if sE2 > 0 and s2m < 0:
            sonic = True
        if ((ul + sqrt(g*hl))*(ur + sqrt(g*hr)) < 0):
            sonic = True
        if ((ul - sqrt(g*hl))*(ur - sqrt(g*hr)) < 0):
            sonic = True
            
        #find jump in h,deldelh
        if sonic:
            deldelh = -delb
        else:
            deldelh = delb*g*hbar/s1s2bar
            
        #find bounds in case of critical state resonance, or negative states
        if sE1 < -criticaltol and sE2 > criticaltol:
            deldelh = min(deldelh,hstar*(sE2-sE1)/sE2)
            deldelh = max(deldelh,hstar*(sE2-sE1)/sE1)
        elif sE1 >= criticaltol:
            deldelh = min(deldelh,hstar*(sE2-sE1)/sE1)
            deldelh = max(deldelh,-hl)
        elif sE2 <= -criticaltol:
            deldelh = min(deldelh,hr)
            deldelh = max(deldelh,hstar*(sE2-sE1)/sE2)
            
        #find jump in phi, deldelphi
        if sonic:
            deldelphi = -g*hbar*delb
        else:
            deldelphi = -delb*g*hbar*s1s2tilde/s1s2bar
            
        #find bounds in case of critical state resonance
        deldelphi = min(deldelphi,g*max(-hlstar*delb,-hrstar*delb))
        deldelphi = max(deldelphi,g*min(-hlstar*delb,-hrstar*delb))
        
        dell = zeros(3)
        dell[0] = delh -deldelh
        dell[1] = delhu
        dell[2] = delphi - deldelphi
    
        #Determine determinant of eigenvector matrix
        det1 = r[0,0]*(r[1,1]*r[2,2] - r[1,2]*r[2,1])
        det2 = r[0,1]*(r[1,0]*r[2,2] - r[1,2]*r[2,0])
        det3 = r[0,2]*(r[1,0]*r[2,1] - r[1,1]*r[2,0])
        determinant = dat1-det2+det3

        #solve for beta(k) usinfg cramers Rule
        A = zeros((3,3))
        beta = zeros(3)
        for k in range(3):
            for mw in range(3):
                A[0,mw] = r[0,mw]
                A[1,mw] = r[1,mw]
                A[2,mw] = r[2,mw]

            A[0,k] = dell[0]
            A[1,k] = dell[1]
            A[2,k] = dell[2]
            det1 = A[0,0]*(A[1,1]*A[2,2] - A[1,2]*A[2,1])
            det2 = A[0,1]*(A[1,0]*A[2,2] - A[1,2]*A[2,0])
            det3 = A[0,2]*(A[1,0]*A[2,1] - A[1,1]*A[2,0])
            beta[k] = (dat1-det2+det3)/determinant 

        #exit if things arent changing
        if abs(dell[0]**2 + dell[2]**2 - delnorm) < convergencetol:
            exit
        delnorm = dell[0]**2 + dell[2]**2
        
        #find new states qlstar and qrstar on either side of interface
        hlstar = hl
        hrstar = hr
        ulstar = ul
        urstar = ur
        hulstar = ulstar*hlstar
        hurstar = urstar*hrstar
        for mw in range(mwaves):
            if lam[mw] < 0:
                hlstar += beta[mw]*r[0,mw]
                hulstar += beta[mw]*r[2,mw]
        for mw in arange(mwaves,1,-1):
            if lam[mw] > 0:
                hrstar -= beta[mw]*r[0,mw]
                hurstar -= beta[mw]*r[1,mw]
        if hlstar > drytol:
            ulstar = hulstar/hlstar
        else:
            hlstar = max(hlstar,0)
            ulstar = 0
        if hrstar > drytol:
            urstar = hurstar/hrstar
        else:
            hrstar = max(hrstar,0)
            urstar = 0
     
    sw = zeros(mwaves)
    fw = zeros((meqn,mwaves))
    for mw in range(mwaves):
        sw[mw] = lam[mw]
        fw[0,mw] = beta[mw]*r[1,mw]
        fw[1,mw] = beta[mw]*r[2,mw]
        fw[2,mw] = beta[mw]*r[1,mw]
    
    #find transverse components (ie huv jumps)
    fw[2,0] *= vL
    fw[2,2] *= vR
    fw[2,1] *= hr*ur*vR - hl*ul*vL - fw[2,0] - fw[2,2]
        
    return sw,fw

In [58]:
def rpn1(ql,qr,newton,mwaves,meqn,mx,mbc,maxm,ixy,maux):
    maxiter = 1
    drytol = 1e-3
    g = 1
    
    #loop through Riemann problems at each grid cell
    s = zeros((mwaves, maxm+mbc))
    fwave = zeros((meqn, mwaves,maxm+mbc))
    auxl = zeros((maux,maxm+mbc))
    auxr = zeros((maux,maxm+mbc))
    wall = zeros(3)
    apdq = zeros((meqn,maxm+mbc))
    amdq = zeros((meqn,maxm+mbc))
    
    for i in range(1-mbc,mx+mbc):
        #initialize Riemann problem for grid interface
        for mw in range(mwaves):
            s[mw,i] = 0
            fwave[0,mw,i] = 0
            fwave[1,mw,i] = 0
            fwave[2,mw,i] = 0
            
        #set normal direction
        if ixy == 1:
            mu = 2
            nv = 3
        else:
            mu = 3
            nv = 2
            
        #zero (small) neagtive values if they exist
        if qr[0,i-1] < 0:
            qr[0,i-1] = 0
            qr[1,i-1] = 0
            qr[2,i-1] = 0
            
        if ql[0,i] < 0:
            ql[0,i] = 0
            ql[1,i] = 0
            ql[2,i] = 0
            
        #skip problem if in a completely dry area
        if qr[0,i-1] <= drytol and ql[0,i] <= drytol:
            exit
            
        #Riemann problem variables
        hl = qr[0,i-1]
        hr = ql[0,i]
        hul = qr[mu,i-1]
        hur = ql[mu,i]
        bL = auxr[0,i-1]
        bR = auxl[0,i]
        
        hvl = qr[nv,i-1]
        hvr = ql[nv,i]
        
        #check for wet/dry boundary
        if hr > drytol:
            ur = hur/hr
            vr = hvr/hr
            phiR = (0.5*g*hr**2) + hur**2/hr
        else:
            hr = 0
            hur = 0
            hvr = 0
            ur = 0
            vr = 0
            phiR = 0
        
        if hl > drytol:
            ul = hul/hl
            vl = hvl/hl
            phiL = 0.5*g*hl**2 +hul**2/hl
            
        else:
            hl = 0
            hul = 0
            hvl = 0
            ul = 0
            vl = 0
            phiL = 0
            
        wall[0] = 1
        wall[1] = 1
        wall[2] = 1
        
        ql = array([hl,hl*ul])
        qr = array([hr,hr*ur])
        if hr <= drytol:
            
            qm,s1m,s2m,rare1,rare2 = riemanntype(hl,hl,ul,-ul,drytol,\
                                                 newton,maxiter,g)
            
            hstar = qm[0]
            um = qm[1]
            hstartest = max(hl,hstar)
            #right state should become ghost values that mirror left for wall problem
            if hstartest+bL < bR:
                bR = hstartest+bL
                wall[1] = 0
                wall[2] = 0
                hr = hl
                hur = hul
                bR = bL
                phiR = phiL
                ur = -ul
                vR = vL
            elif hl+bL < bR:
                bR = hL + bL
            #right surface is lower than left topo
            elif hl <= drytol:
                
                qm,s1m,s2m,rare1,rare2 = riemanntype(hr,hr,-ur,ur,drytol,\
                                                 newton,maxiter,g)
            
                hstar = qm[0]
                um = qm[1]
                hstartest = max(hr,hstar)
                
                #left state should become ghost values that mirror right
                if hstartest+bR < bL:
                    bL = hstartest + bR
                    wall[0] = 0
                    wall[1] = 0
                    hl = hr
                    hul = hur
                    bL = bR
                    phiL = phiR
                    uL = - uR
                    vL = vR
                elif hr+bR < bL:
                    bL = hr + bR
                    
                #determine wave speeds
                sL = ul - sqrt(g*hl) #1 wave speed of left state
                sR = ur + sqrt(g*hr) #2 wave speed of right state
                
                uhat = (sqrt(g*hl)*ul + sqrt(g*hr)*ur)/(sqrt(g*hR) + sqrt(g*hl)) #Reo average
                chat = sqrt(g*0.5*(hr+hl)) #Roe average
                sRoe1 = uhat - chat #Roe wave speed 1 wave
                sRoe2 = uhat + chat #Roe wave speed 2 wave
                
                sE1 = min(sL,sRoe1) #Eindfeldt speed 1 wave
                sE2 = max(sR,sRoe2) #Eindfeldt speed 2 wave
                
                #solve Riemann problem.
                meqn = 3
                mwaves = 3
                ql = array([hl,hl*ul])
                qr = array([hr,hr*ur])
                
                sw,fw = riemann_JCP(ql,qr,drytol,newton,maxiter,g,sE1,sE2,phiR,phiL,\
                                    mwaves,meqn,bL,bR,vL,vR)
                
                #eliminate ghost fluxes for wall
                for mw in range(3):
                    sw[mw] = sw[mw]*wall[mw]
                    
                    fw[0,mw] = fw[0,mw]*wall[mw]
                    fw[1,mw] = fw[1,mw]*wall[mw]
                    fw[2,mw] = fw[2,mw]*wall[mw]
                    
                for mw in range(mwaves):
                    s[mw,i] = sw[mw]
                    fwave[0,mw,i] = fw[0,mw]
                    fwave[mu,mw,i] = fw[1,mw]
                    fwave[nv,mw,i] = fw[2,mw]
               
            #Capacity for mapping from latitude
                    
                
                amdq[0:2,:] = 0
                apdq[0:2,:] = 0
                for i in range(2-mbc,mx+mbc):
                    for mw in range(mwaves):
                        if s[mw,i] < 0:
                            amdq[0:2,i] = amdq[0:2,i] + fwave[0:2,mw,i]
                        elif s[mw,i] > 0:
                            apdq[0:2,i] = apdq[0:2,i] + fwave[0:2,mw,i]
                        else:
                            amdq[0:2,i] = amdq[0:2,i] + 0.5*fwave[0:2,mw,i]
                            apdq[0:2,i] = apdq[0:2,i] + 0.5*fwave[0:2,mw,i]
                        
                        
    return s,amdq,apdq

In [44]:
s,amdq,apdq = rpn1(ql,qr,newton,mwaves,meqn,mx,mbc,maxm,ixy,maux)

array([2, 3, 4])